# PREDICCIÓN NUEVOS TICKETS

Se utiliza para procesar y predecir la sugerencia adecuada de nuevos tickets

# Tareas

1.	Carga la fuente de los nuevos ejemplos
2.	Carga los objetos serializados
3.	Repite el proceso de procesamiento utilizando los objetos cargados
4.	Carga los modelos entrenados (4 modelos)
5.	Calcula la predicción de cada variable dependiente
6.	Prepara y exporta la respuesta


# Importación de librerias

In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 800)

In [3]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import gensim
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
import nltk

C:\ProgramData\Anaconda3\envs\T\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
import json, re
from joblib import dump,load
from datetime import datetime

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

In [6]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """
    # Eliminamos los caracteres especiales
    texto = re.sub(r'\W', ' ', str(texto))
    # Eliminamos comas y /
    texto = re.sub(r',', ' ', str(texto))
    texto = re.sub(r'/', ' ', str(texto))
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    # Convertimos textos a minusculas
    texto = texto.lower()
    return texto

In [7]:
nltk.download('stopwords')
STOPWORDS = set(stopwords.words("spanish"))
def filtrar_stopword_digitos(tokens):
    """
    Filtra stopwords y digitos de una lista de tokens.
    """
    return [token for token in tokens if token not in STOPWORDS 
            and not token.isdigit()]

stemmer = SnowballStemmer("spanish")
def stem_palabras(tokens):
    """
    Reduce cada palabra de una lista dada a su raíz.
    """
    return [stemmer.stem(token) for token in tokens]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
stemmer = SnowballStemmer("spanish")
def stem_palabras(tokens):
    """
    Reduce cada palabra de una lista dada a su raíz.
    """
    return [stemmer.stem(token) for token in tokens]

In [9]:
def Limpiar_Caracter(texto):
    if len(texto) == 1 or texto == ',':
        texto = ''
    else:
        texto = texto
    return texto

In [10]:
#modelo topicos
### Funciones para modelo de tópicos
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def impresionTopicos(mod,nPalabras,nTopicos):
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    for topico in topicos:
        print(topico)    
        
def impresionTopicosLimp(mod,nPalabras,nTopicos):
    #Impresion limpia de Tópicos
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    i=0
    for topico in topicos:
        i=i+1
        textoAux = re.sub(r'\W', ' ', str(topico))
        textoAux = re.sub(r'\s+[a-zA-Z]\s+', ' ', textoAux)
        textoAux = re.sub(r'[0-9]', ' ', textoAux)
        textoAux = re.sub(r'\s+', ' ', textoAux, flags=re.I)
        print(str(i),textoAux)

In [11]:
def disbribucionProbTopicos(row, var):   
    arr_text=row[var]   
    lda=dic_lda[var]
    diccionario=dic_TextDic[var]
    if arr_text==[]:
        return np.zeros(num_topicos)
    else:
        bow_comentario = diccionario.doc2bow(arr_text)
        try:
            distribucion_noticia = lda[bow_comentario]
        except:
            return np.zeros(num_topicos)        
        df_aux=pd.DataFrame(distribucion_noticia,columns=['ind','prob'])        
        df_plan=pd.DataFrame(index=range(0,num_topicos))
        df_plan=df_plan.merge(df_aux, left_index=True, right_on='ind', how='left')
        df_plan.fillna(0,inplace=True)
        df_plan.set_index('ind', inplace=True)          
        return df_plan['prob']

# Carga de Archivos
* maximo.classification.csv
* maximo.classstructure.csv
* maximo.failurecode.csv
* maximo.incident.csv
* maximo.longdescription.csv

# Crear archivos de ejemplo

In [14]:
df_independientes= pd.read_csv('DATOS/maximo.incident_4_tablas_sample.csv',sep=',', header = 0)

In [15]:
len(df_independientes)

100

In [16]:
len(df_independientes[df_independientes['STATUS']=='CERRADO'])

100

In [17]:
len(df_independientes[df_independientes['FAILURECODE'].notnull()])

100

# Cargar objetivos serializados

In [20]:
class_ohe=load(f"OBJETOS/clas_ohe.pkl")
dic_opc_categoricas=load(f"OBJETOS/posiblesCat.pkl")
dic_lda=load(f"OBJETOS/ldas.pkl")

In [21]:
dic_TextDic= load(f"OBJETOS/dic_TextDic.pkl")

In [22]:
#carga de modelos
modelo_failureCode=load(f"OBJETOS/modelo_failureCode.pkl")
modelo_problem=load(f"OBJETOS/modelo_problem.pkl")
modelo_remedy=load(f"OBJETOS/modelo_remedy.pkl")
modelo_cause=load(f"OBJETOS/modelo_cause.pkl")

# Parametros globales

In [23]:
num_min_cat=150
num_topicos=5

# Procesamiento nuevos casos

In [24]:
var_num=['IMPACT','INCCARGAPRIORIDAD','INTERNALPRIORITY','OUTAGEDURATION','URGENCY']
var_texto=['CL_C_DESC_AFECTACION','C_CL_AFECTA','DESCRIPTION','INCMEXCLUSION','INCSOLUCION',
           #agrego por validacion
           'CL_INC_IMPACTO']
var_bool=['DIAGNOSTICADO','HASACTIVITY','HASLD','INCEXCLUIR','INCEXCLUIR_ENLD','INC_ALARMA','MUN100','SRDATOS','SRINTERNET','SRTV','SRVOZ',
         'HASCHILDREN','USECLASSINDESC'
         ]
var_cat=['ASSIGNEDOWNERGROUP','CL_C_TIPO','CL_SERVICIO','CL_SER_AFEC','EXTERNALSYSTEM','FR1CODE','ONCALLOWNERGROUP','ORIGRECORDCLASS','PROBLEMCODE','PROVEEDORES','REGIONAL','TICKET_EXT',
        'TIPOCOMUNICADO','WORKTYPE','FR2CODE','INCSERVICE','CLASSIFICATIONID','DESCRIPCION_TECNICA','INCMESTADO',
         'PERSONGROUP','DESCRIPTION.2'
        ]
var_etiqueta=['FAILURECODE','TICKETID']

In [25]:
df_independientes.set_index('TICKETID',inplace=True)

In [26]:
df_independientes.fillna(-1,inplace=True)

In [27]:
df_num_fin=df_independientes[var_num+var_bool]
df_num_fin

,IMPACT,INCCARGAPRIORIDAD,INTERNALPRIORITY,OUTAGEDURATION,URGENCY,DIAGNOSTICADO,HASACTIVITY,HASLD,INCEXCLUIR,INCEXCLUIR_ENLD,INC_ALARMA,MUN100,SRDATOS,SRINTERNET,SRTV,SRVOZ,HASCHILDREN,USECLASSINDESC
TICKETID,,,,,,,,,,,,,,,,,,
INC1081335,3.0,2.0,3.0,0.000000,3.0,0,0,1,0,0,0,0,0,0,0,0,0.0,1.0
INC4082980,3.0,-1.0,3.0,1.933333,3.0,0,0,1,0,0,0,0,0,0,0,0,0.0,1.0
INC3348348,3.0,-1.0,3.0,19.616667,3.0,1,1,1,0,0,0,0,0,1,0,0,0.0,1.0
INC3099996,3.0,-1.0,3.0,0.000000,3.0,1,1,1,0,0,1,0,0,0,0,0,0.0,1.0
INC5100274,2.0,-1.0,3.0,4.100000,3.0,1,0,1,0,0,0,0,0,0,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INC3885835,2.0,-1.0,3.0,0.000000,3.0,1,0,1,0,0,0,0,0,0,0,0,0.0,1.0
INC1862632,3.0,2.0,3.0,172.683333,3.0,1,1,1,0,0,0,0,0,1,1,1,0.0,1.0
INC1861937,1.0,4.0,1.0,8.116667,1.0,0,1,1,0,0,0,0,0,1,0,1,0.0,1.0


In [28]:
df_cat_fin=df_independientes[var_cat]
df_cat_fin

,ASSIGNEDOWNERGROUP,CL_C_TIPO,CL_SERVICIO,CL_SER_AFEC,EXTERNALSYSTEM,FR1CODE,ONCALLOWNERGROUP,ORIGRECORDCLASS,PROBLEMCODE,PROVEEDORES,REGIONAL,TICKET_EXT,TIPOCOMUNICADO,WORKTYPE,FR2CODE,INCSERVICE,CLASSIFICATIONID,DESCRIPCION_TECNICA,INCMESTADO,PERSONGROUP,DESCRIPTION.2
TICKETID,,,,,,,,,,,,,,,,,,,,,
INC1081335,FRONTOFFICE,-1,-1,-1,-1,DANADO VIDA UTIL,-1,-1,PATCHCORD PIGTAIL R.O V-HUB,-1,CENTRO ORIENTE,-1.0,-1,ME,REEMPLAZAR,-1,GESTION INTERNA,-1.0,-1,FRONTOFFICE,SERVICIOS MOVILES \ NOTIFICACION/POSIBLE AFECTACION/SIN AFECTACION \ GESTION INTERNA
INC4082980,FOINFRAESTRUCTURA,-1,-1,-1,-1,-1,FOINFRAESTRUCTURA,-1,-1,-1,CENTRO ORIENTE,-1.0,-1,CCOAX,-1,-1,ALARMA FUENTE AUSENCIA DE ENERGIA,-1.0,-1,FOINFRAESTRUCTURA,SERVICIOS FIJOS \ FO Y COAX \ NOTIFICACION/POSIBLE AFECTACION/SIN AFECTACION \ ALARMA FUENTE AUSENCIA DE ENERGIA
INC3348348,FRONTCCI,-1,-1,-1,-1,-1,FRONTCCI,SR,-1,-1,CENTRO ORIENTE,-1.0,-1,CCOAX,-1,-1,SENAL DEFICIENTE,-1.0,-1,FRONTCCI,SERVICIOS FIJOS \ RECLAMACION DE USUARIO/PQR/CPD \ VOZ CLIENTE \ TELEVISION/TELEFONIA/INTERNET \ SENAL DEFICIENTE
INC3099996,AMDOCS-TRIAGE 4G,-1,PERFORMANCE,-1,4GKPIE,FALLA_TX,AMDOCS-TRIAGE 4G,-1,ALARMA_POSIBLE_DEGRADACION_4G,-1,OCCIDENTE,-1.0,-1,-1,TICKET_DUPLICADO,PERFORMANCE,OFENSOR AGUDO,-1.0,-1,FOPERFORMANCE,SERVICIOS MOVILES \ PERFORMANCE/CALIDAD/DEGRADACION \ OFENSOR AGUDO
INC5100274,FOENERGI,-1,-1,-1,-1,FALLA_ENER_COM_SITIO SIN PLANTA,FRONTOFFICE,-1,NOT_ENERGIA,AIR-E,COSTA,6565495.0,-1,-1,RETORNA AC COMERCIAL,-1,ALARMA DE ENERGIA,-1.0,-1,FRONTOFFICE,SERVICIOS MOVILES \ NOTIFICACION/POSIBLE AFECTACION/SIN AFECTACION \ ALARMA DE ENERGIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INC3885835,FRONTOFFICE,-1,-1,-1,-1,C_FALLA_POWER,FRONTOFFICE,-1,CORR_ENERGIA,-1,ORIENTE,-1.0,-1,-1,REEMPLAZAR,-1,GESTION DE PLATAFORMAS ELECTROMECANICAS,-1.0,-1,FRONTOFFICE,SERVICIOS MOVILES \ NOTIFICACION/POSIBLE AFECTACION/SIN AFECTACION \ GESTION DE PLATAFORMAS ELECTROMECANICAS
INC1862632,FOHFC,-1,-1,-1,-1,RUIDO MALA CONECTIVIDAD,-1,SR,-1,-1,CENTRO ORIENTE,-1.0,-1,CCOAX,CONECTORIZAR,-1,NODO CON RUIDO / SNR / PL / MOD,-1.0,-1,FOHFC,SERVICIOS FIJOS \ FO Y COAX \ PERFORMANCE/CALIDAD/DEGRADACION \ NODO CON RUIDO / SNR / PL / MOD
INC1861937,FOHFC,AUSENCIA DE SERVICIO,-1,-1,-1,VERIFICACION DE SENAL COAXIAL,-1,-1,-1,-1,CENTRO ORIENTE,-1.0,FALLA MASIVA,CCOAX,VERIFICAR SERVICIO OK,-1,SECTOR SIN SENAL,-1.0,-1,FOHFC,SERVICIOS FIJOS \ FO Y COAX \ AFECTACION DE SERVICIO_ACCESO \ SECTOR SIN SENAL


In [29]:
for var in var_cat:
    df_cat_fin[var]=df_cat_fin[[var]].applymap(str)
    df_cat_fin[var]=df_cat_fin[var].apply(lambda x: re.sub('[^A-Za-z0-9_.]+', '_', x))
    
    #cargar variables principales
    aux_cats=dic_opc_categoricas[var]    
    df_cat_fin[var]=np.where(df_cat_fin[var].isin(aux_cats),df_cat_fin[var],'otro')

In [30]:
tablaAnalisis_class_ohe = class_ohe.transform(df_cat_fin[var_cat]).toarray()
colDummies=class_ohe.get_feature_names(var_cat)

In [31]:
df_cat_finv2=pd.DataFrame(tablaAnalisis_class_ohe,index=df_cat_fin.index, columns=colDummies)
df_cat_finv2

ASSIGNEDOWNERGROUP_AMDOCS  ASSIGNEDOWNERGROUP_AMDOCS_OPTIMIZACION  \
TICKETID                                                                        
INC1081335                        0.0                                     0.0   
INC4082980                        0.0                                     0.0   
INC3348348                        0.0                                     0.0   
INC3099996                        0.0                                     0.0   
INC5100274                        0.0                                     0.0   
...                               ...                                     ...   
INC3885835                        0.0                                     0.0   
INC1862632                        0.0                                     0.0   
INC1861937                        0.0                                     0.0   
INC5169101                        0.0                                     0.0   
INC2704525                        0.0                                     0.0   

            ASSIGNEDOWNERGROUP_AMDOCS_TRIAGE_2G  \
TICKETID                                          
INC1081335                                  0.0   
INC4082980                                  0.0   
INC3348348                                  0.0   
INC3099996                                  0.0   
INC5100274                                  0.0   
...                                         ...   
INC3885835                                  0.0   
INC1862632                                  0.0   
INC1861937                                  0.0   
INC5169101                                  0.0   
INC2704525                                  0.0   

            ASSIGNEDOWNERGROUP_AMDOCS_TRIAGE_4G  \
TICKETID                                          
INC1081335                                  0.0   
INC4082980                                  0.0   
INC3348348                                  0.0   
INC3099996                                  1.0   
INC5100274                                  0.0   
...                                         ...   
INC3885835                                  0.0   
INC1862632                                  0.0   
INC1861937                                  0.0   
INC5169101                                  0.0   
INC2704525                                  0.0   

            ASSIGNEDOWNERGROUP_BACKTXMW_SATEL_PERFORMANCE  \
TICKETID                                                    
INC1081335                                            0.0   
INC4082980                                            0.0   
INC3348348                                            0.0   
INC3099996                                            0.0   
INC5100274                                            0.0   
...                                                   ...   
INC3885835                                            0.0   
INC1862632                                            0.0   
INC1861937                                            0.0   
INC5169101                                            0.0   
INC2704525                                            0.0   

            ASSIGNEDOWNERGROUP_BACK_AUT_PRUEBA  ASSIGNEDOWNERGROUP_BODATOS  \
TICKETID                                                                     
INC1081335                                 0.0                         0.0   
INC4082980                                 0.0                         0.0   
INC3348348                                 0.0                         0.0   
INC3099996                                 0.0                         0.0   
INC5100274                                 0.0                         0.0   
...                                        ...                         ...   
INC3885835                                 0.0                         0.0   
INC1862632                                 0.0                         0.0   
INC1861937                                 0.0          

In [32]:
df_cat_finv2.sum().sum()

2100.0

# Procesamiento Texto

In [33]:
tokenizer = ToktokTokenizer()

In [34]:
df_text_fin=df_independientes[var_texto]
df_text_fin

,CL_C_DESC_AFECTACION,C_CL_AFECTA,DESCRIPTION,INCMEXCLUSION,INCSOLUCION,CL_INC_IMPACTO
TICKETID,,,,,,
INC1081335,-1,-1,FOIP: Se realiza revisión de gráficas de tráfico de datos en PRTG,-1,FOIP: Se realiza revisión de gráficas de tráfico de datos en PRTG,-1.0
INC4082980,-1,-1,FOINF: MR_T3: Fuente - Alarma por falla de energía,-1,OT : 972895\nNODO: 3ah\nMovil: FSV543\nTécnico: johany Saenz\nSupervisor : Javier rozo \nInterventor : luis florez \nDispositivo :Fuente \nUbicación :clle 64 sur #82d-10\nProblema : alarma energia\nCausa : fluido electrico\nSolucion : Se llega ala dirección reportada se evidencia que hay fluido eléctrico se valida la fuente el cual esta con voltage en 86 voltios de salida y 116 voltios de entrada se escala sin maniobra correctiva se valida por o y m por parte de red externa se garantiza gestión de la fuente ok y sin alarmas,-1.0
INC3348348,-1,-1,Canales bajos,-1,"Se llega al 314 donde se valida dispositivo 14x8 que acomete cuenta reportada encontrando niveles desfasados se le hace seguimiento a la red donde se evidencia dispositivos en mal funcionamiento y sulfatados, se realiza cambio y nueva conectorización garantizando niveles en 14 por 8 que acomete cuenta reportada de canal 18 en 13 canal 124 en 19 tx40 RX 15 garantizando niveles sin ninguna variación se garantiza niveles por parte de red externa se garantizan niveles sin ninguna variación",-1.0
INC3099996,-1,-1,MC: PERFORMANCE Degradación del servicio de datos EB LTE NEI.Fortaleza:P5 sector L1 en NEI KPI_LTE_5060J,-1,-1,-1.0
INC5100274,-1,-1,FEE:MPACC_T1: Alarma de power en baterias EB ATL.Playa Mendoza-1 en Atlántico EB SIN PE.,-1,"FEE CIERRE: LVCMC: Se hace seguimiento a la EB ATL.Playa Mendoza-1 reportada encontrando alarmas canceladas por retorno de red comercial, se procede a cerrar el TK.",-1.0
...,...,...,...,...,...,...
INC3885835,-1,-1,BE_CFD_ING_E_T4/NETNUMEN: Rutina Preventiva POWZTE EB CUN.Villagomez/ Resultado (Exitoso),-1,REVISION DE TK,-1.0
INC1862632,-1,-1,REVISION DE RUIDO NODO MPV4D,-1,-1,-1.0
INC1861937,-1,-1,FOHFC:CAG Cm Desenganchados / Nodo 7UT,-1,-1,-1.0


In [35]:
for var in var_texto:
    print(var)    
    print(" preprocesamiento texto:",datetime.now())
    df_text_fin[var] = df_text_fin[var].apply(limpiar_texto)
    df_text_fin[var] = df_text_fin[var].apply(tokenizer.tokenize)
    df_text_fin[var] = df_text_fin[var].apply(filtrar_stopword_digitos)
    df_text_fin[var] = df_text_fin[var].apply(stem_palabras)

CL_C_DESC_AFECTACION
 preprocesamiento texto: 2022-03-31 09:54:35.570303
C_CL_AFECTA
 preprocesamiento texto: 2022-03-31 09:54:35.588216
DESCRIPTION
 preprocesamiento texto: 2022-03-31 09:54:35.603176
INCMEXCLUSION
 preprocesamiento texto: 2022-03-31 09:54:35.704905
INCSOLUCION
 preprocesamiento texto: 2022-03-31 09:54:35.727842
CL_INC_IMPACTO
 preprocesamiento texto: 2022-03-31 09:54:35.873177


In [36]:
var_texto

['CL_C_DESC_AFECTACION',
 'C_CL_AFECTA',
 'DESCRIPTION',
 'INCMEXCLUSION',
 'INCSOLUCION',
 'CL_INC_IMPACTO']

In [37]:
#Distribuir probabilidad
dfs_proc_texto={}
for var in var_texto:
    print(var," probabilidades salida:",datetime.now())    
    for i in range(0,num_topicos):
        df_text_fin[var+'_'+str(i)]=0
    dfs_proc_texto[var]=df_text_fin.apply(lambda row: disbribucionProbTopicos(row,var), axis=1)

CL_C_DESC_AFECTACION  probabilidades salida: 2022-03-31 09:54:35.923711
C_CL_AFECTA  probabilidades salida: 2022-03-31 09:54:35.966217
DESCRIPTION  probabilidades salida: 2022-03-31 09:54:35.987161
INCMEXCLUSION  probabilidades salida: 2022-03-31 09:54:36.704141
INCSOLUCION  probabilidades salida: 2022-03-31 09:54:36.838058
CL_INC_IMPACTO  probabilidades salida: 2022-03-31 09:54:37.275914


In [39]:
# Seleccionar columnas
var_texto_sel=['CL_INC_IMPACTO','DESCRIPTION','INCMEXCLUSION','INCSOLUCION']

In [40]:
df_text_finV2=pd.DataFrame()
for var in var_texto_sel:
    print(var)
    aux_df=pd.DataFrame.from_dict(dfs_proc_texto[var])
    if len(aux_df.columns)==1:
        aux_df=pd.DataFrame(aux_df[0].tolist(), index= aux_df.index)
        
    aux_df.columns=[f"{var}_{s}" for s in list(range(0,num_topicos))]
    if len(df_text_finV2)==0:
        df_text_finV2=aux_df.copy()
    else:
        df_text_finV2=df_text_finV2.merge(aux_df, left_index=True, right_index=True, how='left')

CL_INC_IMPACTO
DESCRIPTION
INCMEXCLUSION
INCSOLUCION


# Union tablas

In [41]:
df_base=df_text_finV2.merge(df_num_fin, left_index=True, right_index=True, how='left')
df_base=df_base.merge(df_cat_finv2,left_index=True, right_index=True, how='left' )

# Predicciones

In [62]:
pred_fc= modelo_failureCode.predict(df_base)
pred_problem=modelo_problem.predict(df_base)
pred_remedy=modelo_remedy.predict(df_base)
pred_cause=modelo_cause.predict(df_base)

In [63]:
#Probabilidades
pred_fc_proba=modelo_failureCode.predict_proba(df_base).max(axis=1)
pred_problem_proba=modelo_problem.predict_proba(df_base).max(axis=1)
pred_remedy_proba=modelo_remedy.predict_proba(df_base).max(axis=1)
pred_cause_proba=modelo_cause.predict_proba(df_base).max(axis=1)

In [64]:
respuesta=pd.DataFrame(index=df_base.index)

In [65]:
respuesta['FAILURECODE']=pred_fc
respuesta['p_FAILURECODE']=pred_fc_proba
respuesta['PROBLEM']=pred_problem
respuesta['p_PROBLEM']=pred_fc_proba
respuesta['REMEDY']=pred_remedy
respuesta['p_REMEDY']=pred_fc_proba
respuesta['CAUSE']=pred_cause
respuesta['p_CAUSE']=pred_cause_proba

In [66]:
respuesta

,FAILURECODE,p_FAILURECODE,PROBLEM,p_PROBLEM,REMEDY,p_REMEDY,CAUSE,p_CAUSE
TICKETID,,,,,,,,
INC1081335,FIBRA OPTICA,0.991351,PATCHCORD PIGTAIL R.O V-HUB,0.991351,PATCHCORD PIGTAIL R.O V-HUB,0.991351,FIBRA OPTICA,0.991351
INC4082980,ACCESO RED COAXIAL,0.993388,SIN FALLA O&M ACCESO,0.993388,SIN FALLA O&M ACCESO,0.993388,ACCESO RED COAXIAL,0.993388
INC3348348,ACCESO RED COAXIAL,0.999202,TAP,0.999202,TAP,0.999202,ACCESO RED COAXIAL,0.999202
INC3099996,DESEMPENO_ACCESO_MOVIL_MESCAL,0.918897,OTRO,0.918897,OTRO,0.918897,DESEMPENO_ACCESO_MOVIL_MESCAL,0.918897
INC5100274,ACCESO_ESTACION_BASE,0.999390,NOT_ENERGIA,0.999390,NOT_ENERGIA,0.999390,ACCESO_ESTACION_BASE,0.999390
...,...,...,...,...,...,...,...,...
INC3885835,ACCESO_ESTACION_BASE,0.997768,CORR_ENERGIA,0.997768,CORR_ENERGIA,0.997768,ACCESO_ESTACION_BASE,0.997768
INC1862632,ACCESO RED COAXIAL,0.979992,RUIDO,0.979992,RUIDO,0.979992,ACCESO RED COAXIAL,0.979992
INC1861937,ACCESO RED COAXIAL,0.969866,SIN FALLA O&M ACCESO,0.969866,SIN FALLA O&M ACCESO,0.969866,ACCESO RED COAXIAL,0.969866


In [67]:
respuesta.to_csv('RESULTADOS/prediccion.csv')